Chicos porfa revisen que está mal en el código, porque me sale error de columnas cuando existen
y antes de eso me tomó la vida entender por que no multiplicaba las matrices,
este codigo lo que hace es tomar dos archivos csv, uno que es rating que es la union de todos los csv
y el otro es peliculas que es el mismo que ustedes tenian ya hecho, basicamente hace multiplicación de matrices el codigo
para generar un patron de reconocimiento de peliculas, les dejo el enlace aca tambien del youtube donde saque

https://www.youtube.com/watch?v=etD6VeGvlZU&t=472s

In [1]:
import pandas as pd
from math import sqrt
import numpy as np

peliculas = pd.read_csv('datasets_norm/registros_peliculas.csv', encoding='utf-8')

# Se descargan los archivos csv para análisis de machine learning

dfCsv1 = pd.read_csv('ratings/1.csv', encoding='utf-8')
dfCsv2 = pd.read_csv('ratings/2.csv', encoding='utf-8')
dfCsv3 = pd.read_csv('ratings/3.csv', encoding='utf-8')
dfCsv4 = pd.read_csv('ratings/4.csv', encoding='utf-8')
dfCsv5 = pd.read_csv('ratings/5.csv', encoding='utf-8')
dfCsv6 = pd.read_csv('ratings/6.csv', encoding='utf-8')
dfCsv7 = pd.read_csv('ratings/7.csv', encoding='utf-8')
dfCsv8 = pd.read_csv('ratings/8.csv', encoding='utf-8')

In [3]:
# Uno los csv que contienen los rating

rating = pd.concat([dfCsv1, dfCsv2, dfCsv3, dfCsv4, dfCsv5, dfCsv6, dfCsv7, dfCsv8], ignore_index=True)

# Ahora vamos a dividir los valores de la columna de géneros en una lista de géneros para simplificar el uso futuro
peliculas['genero'] = peliculas['listed_in'].str.split(',')

for index, row in peliculas.iterrows():
    if isinstance(row['listed_in'], list):
        for genero in row['listed_in']:
            peliculas.at[index, genero] = 1

peliculas = peliculas.fillna(0)

# Eliminamos columnas innecesarias
if 'timestamp' in rating.columns:
    rating = rating.drop('timestamp', axis=1)

# Creo un Perfil aleatorio de usuario para entrenamiento de datos
userId_0 = {
    'title': ['the amazing race', 'man from del rio', 'van helsing', 'love, rosie', 'the big show', 'timbertown', 'bethany hamilton: unstoppable', 'the gathering', 'set up', 'lovely'],
    'rating': [5, 3.5, 5, 2, 2, 4, 0, 0, 0, 0]
}

entrada_peli = pd.DataFrame(userId_0)



In [4]:
# Ahora vamos a agregarle identificación a cada película que ingresó el usuario y eso lo haremos filtrando las filas que
# contienen el título de la película y luego fusionando con nuestra matriz de películas originales.
Id = peliculas[peliculas['title'].isin(entrada_peli['title'].tolist())]  # Filtramos
entrada_peli = pd.merge(Id, entrada_peli, on='title')  # Fusionamos las matrices

# Eliminamos información innecesaria
columns_to_drop = ['listed_in', 'release_year']
entrada_peli = entrada_peli.drop(columns_to_drop, axis=1)

# Ahora vamos a codificar las películas del usuario, para asignar los géneros por medio del método One Hot Encoder
peli_usuario = peliculas[peliculas['show_id'].isin(entrada_peli['show_id'].tolist())]

# Seguimos limpiando información innecesaria
peli_usuario = peli_usuario.reset_index(drop=True)
columns_to_drop = ['show_id', 'title']




In [5]:
# Seguimos limpiando información innecesaria
peli_usuario = peli_usuario.reset_index(drop=True)

columns_to_drop = ['show_id', 'title', 'listed_in', 'release_year']

tabla_generos = peli_usuario.drop(columns_to_drop, axis=1)

In [6]:
# Convertir la columna 'rating_y' a valores numéricos
entrada_peli['rating_y'] = pd.to_numeric(entrada_peli['rating_y'], errors='coerce')

# Llenar los valores no válidos con cero
entrada_peli['rating_y'] = entrada_peli['rating_y'].fillna(0)

# Convertir la columna 'rating_y' a tipo float
entrada_peli['rating_y'] = entrada_peli['rating_y'].astype(float)

# Verificar las dimensiones de las matrices
print(tabla_generos.transpose().shape)  # Debe ser compatible con la forma de entrada_peli['rating_y']

# Realizar la modelación de datos
perfil_usu = np.matmul(tabla_generos.transpose().values, entrada_peli['rating_y'].values)


(10, 12)


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
# Ahora vamos a extraer los géneros de la tabla original
generos = peliculas.set_index('show_id')

##### Mostramos la información necesaria
columns_to_drop = ['title', 'listed_in', 'release_year']
generos = generos.drop(columns_to_drop, axis=1)

# Ahora realizamos el promedio ponderado para recomendar las 20 películas que hayan en nuestra base de datos
recom = ((generos * perfil_usu).sum(axis=1) / perfil_usu.sum())

# Ordenamos las recomendaciones en orden descendente
recom = recom.sort_values(ascending=False)

# Tabla de recomendación final
final = peliculas[peliculas['show_id'].isin(recom.head(20).keys())]
nfinal = final['title']

In [86]:
print(type(entrada_peli['rating_y']))


KeyError: 'rating_y'